In [1]:
#from google.cloud import secretmanager
import pandas as pd
import numpy as np
from currensee.utils.db_utils import create_pg_engine
from sqlalchemy import text

import pandas as pd
import random
import os
import re



2025-07-05 19:26:51,351 - currensee.core.secrets - INFO - PROJECT_ID not found, attempting to load from .env file
2025-07-05 19:26:51,361 - currensee.core.secrets - INFO - Loaded environment from ./.env
2025-07-05 19:26:51,363 - currensee.core.secrets - INFO - SecretManager initialized with project_id: adsp-34002-on02-sopho-scribe


In [2]:
# Define DB_NAME
DB_NAME = "crm_outlook"

In [3]:
engine = create_pg_engine(db_name=DB_NAME)

In [4]:
pd.read_sql("SELECT * FROM preferences limit 10", con=engine)

,as_of_date,employee_first_name,employee_last_name,title,finance_detail,news_detail,macro_news_detail,past_meeting_detail
0,2023-01-23 10:00:00,Jane,Moneypenny,Relationship Manager,True,False,False,True
1,2023-01-23 10:00:00,Monica,Melton,Product Specialist,False,False,True,False
2,2023-01-23 10:00:00,Michael,Long,Finance Assistant,False,True,False,False
3,2023-01-23 10:00:00,Breanna,Guerra,Senior Relationship Manager,True,True,False,False
4,2023-01-23 10:00:00,Heather,Singh,Financial Advisor,True,False,False,False
5,2023-01-23 10:00:00,Alejandro,Walters,Senior Relationship Manager,True,True,False,False
6,2023-01-23 10:00:00,Ryan,Holloway,Relationship Manager,True,True,False,True
7,2023-01-23 10:00:00,Anthony,Reynolds,Financial Advisor,True,False,True,True
8,2023-01-23 10:00:00,Alexandra,Terry,Product Specialist,False,True,False,False
9,2023-01-23 10:00:00,Jordan,Robles,Financial Advisor,True,False,False,True


In [6]:
pd.read_sql("""SELECT as_of_date
, employee_first_name
, employee_last_name
, finance_detail
, news_detail
, macro_news_detail
, past_meeting_detail 
FROM preferences 
where employee_first_name = 'Jane' and employee_last_name = 'Moneypenny' 
""", con=engine)

,as_of_date,employee_first_name,employee_last_name,finance_detail,news_detail,macro_news_detail,past_meeting_detail
0,2023-01-23 10:00:00,Jane,Moneypenny,True,False,False,True
1,2023-04-02 10:00:00,Jane,Moneypenny,True,False,True,False
2,2023-11-10 10:00:00,Jane,Moneypenny,True,True,True,True
3,2024-01-30 10:00:00,Jane,Moneypenny,True,True,False,True


In [7]:
df = pd.read_sql("SELECT * from employees", con=engine)
df.head()

,employee_id,first_name,last_name,title,email,phone,hire_date,department,market
0,31e735f6-cfbe-4107-8aba-4ef45a90e9ae,Jane,Moneypenny,Relationship Manager,jane.moneypenny1@bankwell.com,971.841.4711x92417,2021-01-14,Enterprise Investment,San Fransisco
1,b61a7a99-791a-4b9f-923f-271810defac4,Monica,Melton,Product Specialist,monica.melton@bankwell.com,884.541.7837,2021-12-03,Small Business Investment,Chicago
2,2f8c004d-e0a6-4bba-956e-2b60af632674,Michael,Long,Finance Assistant,michael.long@bankwell.com,478-831-1994x378,2021-12-06,Sales,Washington DC
3,59dcd404-9d5c-47f4-9ba1-2de5156343f8,Breanna,Guerra,Senior Relationship Manager,breanna.guerra@bankwell.com,736-597-4792x589,2024-07-11,Enterprise Investment,Boston
4,7a650dd8-34a3-4148-a7f8-631bf5db37d4,Heather,Singh,Financial Advisor,heather.singh@bankwell.com,7139112404,2020-11-15,Customer Support,Washington DC


In [10]:
df = pd.read_sql("""SELECT p.as_of_date
        , p.employee_first_name
        , p.employee_last_name
        , p.finance_detail
        , p.news_detail
        , p.macro_news_detail
        , p.past_meeting_detail 
        , e.email	
        FROM preferences p
        inner join employees e
        on p.employee_first_name = e.first_name 
        and p.employee_last_name = e.last_name 
        where e.email = 'jane.moneypenny1@bankwell.com'
        """, con=engine)
df.head()

,as_of_date,employee_first_name,employee_last_name,finance_detail,news_detail,macro_news_detail,past_meeting_detail,email
0,2023-01-23 10:00:00,Jane,Moneypenny,True,False,False,True,jane.moneypenny1@bankwell.com
1,2023-04-02 10:00:00,Jane,Moneypenny,True,False,True,False,jane.moneypenny1@bankwell.com
2,2023-11-10 10:00:00,Jane,Moneypenny,True,True,True,True,jane.moneypenny1@bankwell.com
3,2024-01-30 10:00:00,Jane,Moneypenny,True,True,False,True,jane.moneypenny1@bankwell.com


In [15]:
user_email = df["email"][0]
user_email

'jane.moneypenny1@bankwell.com'

In [22]:

mx_dt_df['as_of_date'][0]

'2024-01-30 10:00:00'

In [23]:
user_email = df["email"][0]

mx_dt_df = pd.read_sql(f"""
    SELECT max(p.as_of_date) as as_of_date
    FROM preferences p
    inner join employees e
    on p.employee_first_name = e.first_name 
    and p.employee_last_name = e.last_name 
    where e.email = '{user_email}'
""", con=engine)

max_dt = mx_dt_df['as_of_date'][0]

query_str = f"""
    SELECT p.as_of_date
    , p.employee_first_name
    , p.employee_last_name
    , p.finance_detail
    , p.news_detail
    , p.macro_news_detail
    , p.past_meeting_detail 
    , e.email	
    FROM preferences p
    inner join employees e
    on p.employee_first_name = e.first_name 
    and p.employee_last_name = e.last_name 
    where e.email = '{user_email}' and p.as_of_date = '{max_dt}'
"""

pref_df = pd.read_sql(query_str, con=engine)


In [25]:
pref_df.head()

,as_of_date,employee_first_name,employee_last_name,finance_detail,news_detail,macro_news_detail,past_meeting_detail,email
0,2024-01-30 10:00:00,Jane,Moneypenny,True,True,False,True,jane.moneypenny1@bankwell.com


In [28]:
fin_detail = pref_df["finance_detail"][0]
news_detail = pref_df["news_detail"][0]
macro_news_detail = pref_df["macro_news_detail"][0]
past_meeting_detail = pref_df["past_meeting_detail"][0]

print(fin_detail, news_detail, macro_news_detail, past_meeting_detail)

True True False True
